# Run the other two notebooks first before running this!

This notebook assumes that you've 1) downloaded the initial dataset, and 2) ran the Branded Food Initial Data Cleaning notebook. Running this with pgadmin will populate the dataset with the cleaned datafiles.

### Tables to create:
- branded_food_reduced -> branded_foods table (main food data file) (done)
- brand_names -> brand_names table (normalization for brand names) (done)
- brand_owners -> brand_owners table (done)
- food_categories -> food_categories table (done)
- ingredients -> ingredients table
- food_to_id -> f_to_ing table (food to ingredient for ingredients, should be made after ingredients)

In [1]:
import os
import psycopg2
import pandas as pd
import psycopg2.extras as extras 
from psycopg2.extensions import register_adapter, AsIs
from sql_creation import (run_command, 
    connect, 
    create_insert_branded_foods,
    create_brand_names,
    create_brand_owners,
    create_food_categories,
    create_ingredients,
    create_food_to_ing,
    create_nutrients,
    create_food_nutrients)
import numpy as np

In [3]:
# Data type handling for numpy numbers
def addapt_numpy_float64(numpy_float64):
    return AsIs(numpy_float64)
def addapt_numpy_int64(numpy_int64):
    return AsIs(numpy_int64)
register_adapter(np.float64, addapt_numpy_float64)
register_adapter(np.int64, addapt_numpy_int64)
register_adapter(np.int32, addapt_numpy_int64)
register_adapter(np.float32, addapt_numpy_float64)

In [4]:
# Create initial connection to the database
conn, cursor = connect()
conn, cursor

Attempting to connect to postgreSQL database...
Connected
postgreSQL db version: ('PostgreSQL 16.2, compiled by Visual C++ build 1937, 64-bit',)


(<connection object at 0x000001E304D329D0; dsn: 'user=postgres password=xxx dbname=food_db host=localhost', closed: 0>,
 <cursor object at 0x000001E31D96D820; closed: 0>)

In [4]:
create_insert_branded_foods(conn, clear_table=True)
conn.commit()

C:\Users\Alexander\Desktop\FDA-Data-Analysis\sql_creation.py:57: DtypeWarning: Columns (2,5,9,10,31) have mixed types. Specify dtype option on import or set low_memory=False.
  reduced_food = pd.read_csv(os.path.join('cleaned', 'branded_food_reduced.csv'))#.drop(columns='Unnamed: 0')


<class 'psycopg2.errors.DuplicateTable'> relation "branded_foods" already exists



In [16]:
create_brand_names(conn)
conn.commit()

In [17]:
create_brand_owners(conn)
conn.commit()

In [18]:
create_food_categories(conn)
conn.commit()

In [19]:
create_ingredients(conn)
conn.commit()

In [57]:
create_food_to_ing(conn)

In [58]:
conn.commit()

In [59]:
create_nutrients(conn)

In [60]:
conn.commit()

In [70]:
create_food_nutrients(conn)

In [71]:
conn.commit()